<a href="https://colab.research.google.com/github/musicjae/Pytorch/blob/main/pytorch_lightning/1_Intro_to_pytorch_lightening.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install pytorch-lightning

In [8]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.core.lightning import LightningModule


## Data

In [9]:
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
import os
from torchvision import datasets, transforms
from pytorch_lightning import Trainer
from torch.optim import Adam


In [10]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,),(0.3081,))])

dataset = MNIST(os.getcwd(), train=True,download=True,transform=transforms.ToTensor()) # os.getcwd는 현재 자신의 디렉터리 위치를 돌려준다.
mnist_test = MNIST(os.getcwd(), train=False,download=True,transform=transforms.ToTensor())

train_loader, val_loader = random_split(dataset, [55000,5000]) #  torch.Size([60000, 28, 28])

In [11]:
train_loader = DataLoader(train_loader,batch_size = 32)
val_loader = DataLoader(val_loader, batch_size = 32)
test_loader = DataLoader(mnist_test, batch_size=32)


## model

In [12]:
class lightening_mnist(pl.LightningModule):
    def __init__(self, lr_rate):
        super().__init__() # super 안이 비어있다는 점 차이
        self.layer1 = torch.nn.Linear(28*28,64)#mnist img(28,28)
        self.layer2 = torch.nn.Linear(64,64)
        self.layer3 = torch.nn.Linear(64,10)
        self.lr_rate = lr_rate

    def forward(self,x):
        batch_size, channels, width, height = x.size()

        x = x.view(batch_size,-1) # # (b, 1, 28, 28) -> (b, 1*28*28)
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.log_softmax(self.layer3(x))

        return x

    def training_step(self,batch,batch_idx):
        x,y = batch
        logits = self.forward(x)
        loss = F.nll_loss(logits,y)
        self.log('train_loss',loss)
        return loss

    def validation_step(self,batch,batch_idx):
        x,y = batch
        logits = self.forward(x)
        loss = F.nll_loss(logits,y)
        self.log('val_loss',loss)
        return loss

    def configure_optimizers(self):
        # self.hparams available because we called self.save_hyperparameters()
        return torch.optim.Adam(self.parameters(), lr=self.lr_rate)


In [22]:
net = lightening_mnist()
net.summarize()


  | Name   | Type   | Params
----------------------------------
0 | layer1 | Linear | 50.2 K
1 | layer2 | Linear | 4.2 K 
2 | layer3 | Linear | 650   
----------------------------------
55.1 K    Trainable params
0         Non-trainable params
55.1 K    Total params


  | Name   | Type   | Params
----------------------------------
0 | layer1 | Linear | 50.2 K
1 | layer2 | Linear | 4.2 K 
2 | layer3 | Linear | 650   
----------------------------------
55.1 K    Trainable params
0         Non-trainable params
55.1 K    Total params

### Trainer

In [23]:
class Trainer:
    def __init__(self,max_epochs=10):
        self.max_epochs = max_epochs

    def _train(self,model,train_loader):
        for epoch in range(self.max_epochs):
            if epoch % 10:
                print(f'epoch:: {epoch}\t loss:: {loss}')
            for batch in train_loader:
                on_batch_strat
                x,y = batch
                prediction = model(x)
                loss = F.nll_loss(prediction,y)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

    def fit(self, model, train_loader):
        self._train(model,train_loader)


In [15]:
model = lightening_mnist(lr_rate = 1e-3)
trainer = pl.Trainer(max_epochs=3,gpus=0,profiler=True)
trainer.fit(model, train_loader,val_loader)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: False
TPU available: None, using: 0 TPU cores
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: GPU available but not used. Set the --gpus flag when calling the script.
  warnings.warn(*args, **kwargs)

  | Name   | Type   | Params
----------------------------------
0 | layer1 | Linear | 50.2 K
1 | layer2 | Linear | 4.2 K 
2 | layer3 | Linear | 650   
----------------------------------
55.1 K    Trainable params
0         Non-trainable params
55.1 K    Total params


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


Profiler Report

Action                      	|  Mean duration (s)	|Num calls      	|  Total time (s) 	|  Percentage %   	|
-----------------------------------------------------------------------------------------------------------------------------
Total                       	|  -              	|_              	|  10.738         	|  100 %          	|
-----------------------------------------------------------------------------------------------------------------------------
run_training_epoch          	|  10.615         	|1              	|  10.615         	|  98.857         	|
get_train_batch             	|  0.0061431      	|1033           	|  6.3458         	|  59.099         	|
run_training_batch          	|  0.0026255      	|1033           	|  2.7122         	|  25.259         	|
optimizer_

1